<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1


    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65


    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-17 12:24:10
-------------------
qualified stocks: 88
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  2.18 L
Current:  1.43 C
-------------------
Today PnL: 55.38 K (0.39%)
Current PnL: -16.56 L (-11.14%)
CY Booked + Current PnL: -5.53 L (-3.72%)
-------------------
Total profit:  2.31 L
Total loss:  -18.87 L
-------------------
Total Booked + Current PnL: 21.45 L (17.62%)
Total Booked PnL: 38.01 L (31.23%)
Curr Year Booked PnL: 11.03 L (7.73%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 82.46 L (57.81%)
Deployed:  1.22 C
Current:  1.43 C
CAGR/XIRR %: 9.85%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SBIN,863.00,-9.44,73.0,M-LC,4.35,223342.0,21862.0,5360.0,1.35,10.85,2.40,13.51,60.0,4.08,1.59,24.81,XY25,NTT,BANKS
5,APOLLOHOSP,8285.00,-13.31,59.0,H-LC,10.96,173272.0,17095.0,8993.0,0.30,10.95,5.19,16.71,22.0,1.90,1.23,30.29,X40N,BTT,HEALTHCARE
78,TTKPRESTIG,770.00,106.19,62.0,M-SC,4.26,88818.0,-11959.0,12053.0,-0.02,-11.87,13.57,0.09,245.0,-0.99,0.63,16.69,OX40N,NTT,DURABLES
50,NATIONALUM,247.44,-42.83,69.0,H-MC,6.32,114411.0,13907.0,16727.0,-0.28,13.84,14.62,30.49,79.0,0.83,0.81,50.73,MH,ATH,METALS
57,RELIANCE,1533.00,-12.16,58.0,H-LC,5.33,220116.0,9750.0,19040.0,0.41,4.63,8.65,13.68,37.0,0.51,1.57,21.92,XY25,NTT,REFINERIES


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,ICICIPRULI,790.00,-21.11,43.0,H-MC,2.27,135837.0,49.0,42707.0,-0.44,0.04,31.44,31.48,107.0,0.00,0.97,12.16,X40,ATH,INSURANCE
30,HAVELLS,2069.16,0.48,60.0,H-MC,1.84,249023.0,1292.0,73761.0,-1.15,0.52,29.62,30.30,92.0,0.02,1.77,13.88,X40,ATH,ELECTRICAL
52,PGHH,17973.08,-30.41,55.0,H-MC,4.28,201450.0,630.0,68151.0,0.46,0.31,33.83,34.25,80.0,0.01,1.43,5.53,X40,ATH,FMCG


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,VBL,671.64,-16.13,42.0,H-LC,6.07,301902.0,-13940.0,126618.0,2.23,-4.41,41.94,35.67,5.0,-0.11,2.15,8.83,X40N,ATH,FMCG
6,ASIANPAINT,3465.66,-13.32,45.0,H-LC,7.97,218891.0,-32877.0,86090.0,0.26,-13.06,39.33,21.13,27.0,-0.38,1.56,17.74,X40,ATH,PAINTS
20,CIPLA,1795.00,-20.12,49.0,H-LC,5.84,212692.0,8192.0,33222.0,-0.42,4.01,15.62,20.25,10.0,0.25,1.51,13.09,X40N,BTT,PHARMA
42,ITC,452.00,-37.92,50.0,H-LC,2.58,200906.0,768.0,20131.0,-0.54,0.38,10.02,10.44,4.0,0.04,1.43,5.97,X40,NTT,FMCG
44,JIOFIN,387.00,0.55,53.0,H-LC,13.32,273136.0,5771.0,59680.0,0.68,2.16,21.85,24.48,48.0,0.10,1.94,58.32,XY24,BTT,FINANCE


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,-1.73,65.0,H-LC,1.06,243608.0,-2543.0,81755.0,0.71,-1.03,33.56,32.18,16.0,-0.03,1.73,34.64,X200,ATH,IT
42,ITC,452.0,-37.92,50.0,H-LC,2.58,200906.0,768.0,20131.0,-0.54,0.38,10.02,10.44,4.0,0.04,1.43,5.97,X40,NTT,FMCG
51,NESTLEIND,1377.0,-8.99,54.0,H-LC,4.06,280145.0,14719.0,42078.0,-0.58,5.55,15.02,21.40,11.0,0.35,1.99,13.03,XY25,NTT,FMCG
57,RELIANCE,1533.0,-12.16,58.0,H-LC,5.33,220116.0,9750.0,19040.0,0.41,4.63,8.65,13.68,37.0,0.51,1.57,21.92,XY25,NTT,REFINERIES
20,CIPLA,1795.0,-20.12,49.0,H-LC,5.84,212692.0,8192.0,33222.0,-0.42,4.01,15.62,20.25,10.0,0.25,1.51,13.09,X40N,BTT,PHARMA


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-39.39,56.0,M-SC,40.35,60258.0,-4748.0,155677.0,3.54,-7.30,258.35,232.18,198.0,-0.03,0.43,4.94,XY24,NTT,MISC
17,CAMPUS,393.00,-22.96,65.0,M-SC,1.36,160572.0,-7203.0,63041.0,2.66,-4.29,39.26,33.28,210.0,-0.11,1.14,24.45,XY24,NTT,FOOTWEAR
70,SURYODAY,240.00,44.74,57.0,H-SC,2.74,132712.0,-46359.0,112089.0,2.28,-25.89,84.46,36.71,135.0,-0.41,0.94,31.68,XR,NTT,BANKS
84,VBL,671.64,-16.13,42.0,H-LC,6.07,301902.0,-13940.0,126618.0,2.23,-4.41,41.94,35.67,5.0,-0.11,2.15,8.83,X40N,ATH,FMCG
56,RELAXO,1176.00,-31.16,60.0,H-SC,1.55,93878.0,-51282.0,127214.0,2.16,-35.33,135.51,52.31,136.0,-0.40,0.67,25.66,X40N,NTT,FOOTWEAR


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,INDIAMART,4850.92,-51.47,45.0,H-SC,12.00,135553.0,12217.0,121550.0,-1.55,9.91,89.67,108.45,119.0,0.10,0.97,34.47,AR,ATH,MISC
22,COFFEEDAY,80.00,-41.65,53.0,L-SC,42.67,85690.0,-27859.0,67909.0,-1.41,-24.53,79.25,35.27,268.0,-0.41,0.61,109.73,XR,NTT,HOTELS
55,RAJOOENG,143.33,-35.10,41.0,H-SC,11.08,98240.0,-4260.0,45092.0,-1.29,-4.16,45.90,39.83,114.0,-0.09,0.70,2.69,AR,ATH,MISC
86,WHIRLPOOL,2270.00,-40.41,53.0,M-SC,0.47,100815.0,9316.0,69431.0,-1.29,10.18,68.87,86.07,223.0,0.13,0.72,46.40,XR,NTT,DURABLES
30,HAVELLS,2069.16,0.48,60.0,H-MC,1.84,249023.0,1292.0,73761.0,-1.15,0.52,29.62,30.30,92.0,0.02,1.77,13.88,X40,ATH,ELECTRICAL


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,106.19,62.0,M-SC,4.26,88818.0,-11959.0,12053.0,-0.02,-11.87,13.57,0.09,245.0,-0.99,0.63,16.69,OX40N,NTT,DURABLES
79,UJJIVANSFB,60.0,96.03,55.0,H-SC,12.36,123768.0,-18711.0,38232.0,-0.28,-13.13,30.89,13.70,163.0,-0.49,0.88,46.08,OX40N,NTT,BANKS
45,KANSAINER,340.0,-67.61,59.0,H-SC,1.71,223506.0,-46161.0,82496.0,1.33,-17.12,36.91,13.47,138.0,-0.56,1.59,13.05,XY24,NTT,PAINTS
19,CERA,9475.0,-20.05,46.0,H-SC,2.15,145670.0,-30233.0,72252.0,-0.09,-17.19,49.60,23.89,149.0,-0.42,1.04,25.81,OX40N,NTT,CERAMICS
73,TATAELXSI,9161.0,-19.85,53.0,H-MC,7.77,108509.0,-23326.0,65550.0,0.07,-17.69,60.41,32.03,98.0,-0.36,0.77,21.54,OX40N,NTT,IT


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4476.75,-25.15,59.0,H-LC,12.58,298036.0,-47920.0,122791.0,0.79,-13.85,41.20,21.64,1.0,-0.39,2.12,5.58,X40,ATH,IT
40,INFY,2275.00,-16.73,55.0,H-LC,8.66,324165.0,11277.0,160397.0,0.70,3.60,49.48,54.87,3.0,0.07,2.31,10.49,X40,BTT,IT
42,ITC,452.00,-37.92,50.0,H-LC,2.58,200906.0,768.0,20131.0,-0.54,0.38,10.02,10.44,4.0,0.04,1.43,5.97,X40,NTT,FMCG
84,VBL,671.64,-16.13,42.0,H-LC,6.07,301902.0,-13940.0,126618.0,2.23,-4.41,41.94,35.67,5.0,-0.11,2.15,8.83,X40N,ATH,FMCG
1,ABB,7934.00,-38.41,64.0,H-LC,10.76,257582.0,-4037.0,123253.0,0.32,-1.54,47.85,45.57,7.0,-0.03,1.83,9.60,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,45.15,66.0,M-MC,6.96,229198.0,4236.0,163418.0,0.12,1.88,71.30,74.53,192.0,0.03,1.63,34.74,XY24,BTT,STEEL
15,BLUESTARCO,2326.38,12.23,61.0,H-MC,3.57,193670.0,29000.0,38966.0,0.65,17.61,20.12,41.28,89.0,0.74,1.38,27.00,X40N,ATH,AC
85,VOLTAS,1918.49,0.74,57.0,H-MC,2.56,211215.0,19473.0,76565.0,-0.46,10.16,36.25,50.08,99.0,0.25,1.50,17.64,XY25,ATH,AC
30,HAVELLS,2069.16,0.48,60.0,H-MC,1.84,249023.0,1292.0,73761.0,-1.15,0.52,29.62,30.30,92.0,0.02,1.77,13.88,X40,ATH,ELECTRICAL
18,CAMS,5211.76,-2.73,59.0,H-SC,4.23,110692.0,8688.0,35233.0,1.62,8.52,31.83,43.06,122.0,0.25,0.79,27.09,X40N,ATH,MISC


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,247.44,-42.83,69.0,H-MC,6.32,114411.0,13907.0,16727.0,-0.28,13.84,14.62,30.49,79.0,0.83,0.81,50.73,MH,ATH,METALS
86,WHIRLPOOL,2270.00,-40.41,53.0,M-SC,0.47,100815.0,9316.0,69431.0,-1.29,10.18,68.87,86.07,223.0,0.13,0.72,46.40,XR,NTT,DURABLES
37,INDIAMART,4850.92,-51.47,45.0,H-SC,12.00,135553.0,12217.0,121550.0,-1.55,9.91,89.67,108.45,119.0,0.10,0.97,34.47,AR,ATH,MISC
87,WIPRO,420.00,-10.16,58.0,M-LC,5.58,158044.0,7099.0,102349.0,0.35,4.70,64.76,72.51,53.0,0.07,1.13,11.01,XR,NTT,IT
36,IEX,219.00,-28.54,60.0,H-SC,17.12,204052.0,4098.0,93129.0,2.03,2.05,45.64,48.63,115.0,0.04,1.45,13.84,XR,NTT,MISC


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RAJESHEXPO,518.00,1897.47,67.0,L-SC,3.43,54497.0,-82680.0,82770.0,1.05,-60.27,151.88,0.07,267.0,-1.00,0.39,35.54,OX40N,NTT,JEWELLERY
22,COFFEEDAY,80.00,-41.65,53.0,L-SC,42.67,85690.0,-27859.0,67909.0,-1.41,-24.53,79.25,35.27,268.0,-0.41,0.61,109.73,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-3.33,51.0,H-SC,13.01,90016.0,-10947.0,105481.0,0.91,-10.84,117.18,93.63,148.0,-0.10,0.64,34.08,SR,ATH,CHEMICALS
49,MASFIN,398.61,-16.44,55.0,H-SC,12.97,95100.0,-2880.0,24479.0,-0.83,-2.94,25.74,22.05,152.0,-0.12,0.68,38.23,XR,ATH,FINANCE
50,NATIONALUM,247.44,-42.83,69.0,H-MC,6.32,114411.0,13907.0,16727.0,-0.28,13.84,14.62,30.49,79.0,0.83,0.81,50.73,MH,ATH,METALS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,ATULAUTO,844.0,3867.57,63.0,M-SC,6.15,123711.0,-23657.0,75476.0,-1.08,-16.05,61.01,35.16,236.0,-0.31,0.88,26.36,XY24,NTT,AUTO
22,COFFEEDAY,80.0,-41.65,53.0,L-SC,42.67,85690.0,-27859.0,67909.0,-1.41,-24.53,79.25,35.27,268.0,-0.41,0.61,109.73,XR,NTT,HOTELS
54,RAJESHEXPO,518.0,1897.47,67.0,L-SC,3.43,54497.0,-82680.0,82770.0,1.05,-60.27,151.88,0.07,267.0,-1.00,0.39,35.54,OX40N,NTT,JEWELLERY
27,GREENPANEL,537.0,238.50,66.0,M-SC,3.45,151060.0,-32018.0,110999.0,1.01,-17.49,73.48,43.14,230.0,-0.29,1.08,42.09,XY24,NTT,MISC
82,VAIBHAVGBL,521.0,64.74,71.0,H-SC,7.98,142698.0,-40077.0,151146.0,1.39,-21.93,105.92,60.77,125.0,-0.27,1.02,27.81,XR,NTT,JEWELLERY


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.48
1,25,42.75
2,50,73.18


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.30
LC    34.02
MC    22.65
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.50
X40      15.00
X40N     12.83
XR       11.42
XY25      9.58
AR        8.52
OX40N     8.01
X200      1.73
MH        1.68
X5K       1.43
SR        1.27
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    28.46
H-LC    25.86
H-MC    19.75
M-SC    13.32
M-LC     7.11
M-MC     2.60
L-SC     1.52
L-LC     1.05
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.23,-13.16,74.86
FMCG,11.57,-4.10,40.53
FINANCE,9.49,-15.37,60.56
BANKS,7.66,-11.36,62.62
MISC,7.35,-11.40,73.36
PAINTS,5.71,-15.08,32.23
ELECTRICAL,5.59,-4.78,62.53
AC,3.58,3.70,28.91
INSURANCE,3.36,-6.48,43.00


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2841356.0,22
XR,1306807.0,14
AR,1299455.0,9
X40,741828.0,10
X40N,610133.0,10
OX40N,520749.0,10
XY25,404558.0,7
SR,243524.0,2
X5K,117147.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3413958.0,29
M-SC,1376217.0,17
H-LC,1239762.0,15
H-MC,1176876.0,15
M-LC,381547.0,5
M-MC,312845.0,2
L-SC,247695.0,3
L-MC,60498.0,1
L-LC,36577.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1029662.0      6
M-SC       XY24       795774.0      7
H-SC       AR         783710.0      5
           XR         754614.0      7
H-MC       XY24       486704.0      4
H-LC       X40        461525.0      5
           AR         354349.0      2
M-MC       XY24       312845.0      2
H-SC       X40N       290160.0      4
           OX40N      250352.0      4
           SR         243524.0      2
H-LC       X40N       221335.0      4
H-MC       X40        211301.0      4
           XY25       187500.0      2
L-SC       XR         164925.0      2
M-SC       AR         161396.0      2
M-LC       XY24       156691.0      2
M-SC       OX40N      122077.0      4
M-LC       X5K        117147.0      1
M-SC       XY25       114003.0      1
           XR         113965.0      2
H-MC       XR         110456.0      1
M-LC       XR         102349.0      1
H-MC       X40N        98638.0      2
L-SC       OX40N       82770.0      1
H-LC       X200        81755.0      1
M-SC       X40         69002.0      1
H-MC       OX40N       65550.0      1
H-SC       MH          61936.0      1
H-LC       XY25        61118.0      2
L-MC       XR          60498.0      1
H-LC       XY24        59680.0      1
L-LC       XY25        36577.0      1
H-MC       MH          16727.0      1
M-LC       XY25         5360.0      1

In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
